# Workshop Outline: Predicting Turbine Loads with Neural Networks

This workshop will guide you through the steps of using a neural network (MLP) to predict turbine Loads from SCADA data.

---

## Steps

1. **Load SCADA Data**  
   Load 10-minute statistics of preprocessed SCADA turbine data.

2. **Explore and Visualize Parameters**  
   Visualize key SCADA parameters such as windspeed, pitch, and power.

3. **Train a Neural Network (MLP)**  
   Train a simple predefined neural network architecture to predict turbine Loads.

4. **Plot and Interpret Performance**  
   Evaluate the network with training and validation error plots, and visualize predictions.

5. **Experiment with Model Complexity**  
   Incrementally increase model complexity and add features to observe changes in performance.

---

**Goal:** Understand the workflow of machine learning for turbine Load prediction, from data exploration to model evaluation and experimentation.


This workshop utilizes open source data from [Aventa AV-7 (6kW) IET-OST Research Wind Turbine SCADA](https://zenodo.org/records/17362783). The original dataset is 1Hz SCADA but is preprocessed in 10 minute SCADA statistics.

# *Loading Necessary Packages*

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Packages for handling and processing load data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Loading Data

In [ ]:
url = "https://raw.githubusercontent.com/OWI-Lab/AI_for_SHM-Workshop/main/Notebooks/AV-7-10min-Data.parquet"
df = pd.read_parquet(url)

df.dropna(inplace=True)
df.head()

# Define damage function 

In [ ]:
def damage_func(row,
                v_cut=25.0,        # ~ cut-out wind speed [m/s]
                power_ref=3000.0,  # ~ rated power [kW], change if needed
                rotor_ref=20.0,    # ~ rated rotor speed [rpm]
                temp_ref=60.0):    # ref generator temp [°C]
    # --- raw inputs ---
    v      = row['wind_speed-mean']
    v_std  = row['wind_speed-std']
    P      = row['power_output-mean']          # in kW (adjust if you use MW)
    rotor  = row['rotor_speed-mean']
    yaw    = row['relative_wind_direction-mean']   # deg
    pitch  = row['blade_pitch_deg-mean']           # deg
    T      = row['generator_temperature-mean']

    # --- simple normalizations (clipped) ---
    v_n     = np.clip(v / v_cut, 0.0, 2.0)        # allow up to 2x for storms
    P_n     = np.clip(P / power_ref, 0.0, 2.0)
    rotor_n = np.clip(rotor / rotor_ref, 0.0, 2.0)
    yaw_n   = np.clip(abs(yaw) / 45.0, 0.0, 2.0)  # 45° misalignment ~ 1
    pitch_n = np.clip(pitch / 80.0, 0.0, 2.0)     # 80° ~ feathered

    # --- helper: smooth indicators ---
    def sigmoid(x):
        return 1.0 / (1.0 + np.exp(-x))

    # operating: high when power is non-zero
    I_op = sigmoid(5.0 * (P_n - 0.05))

    # parked in high wind: high wind but not producing
    I_park = sigmoid((v - 10.0) / 2.0) * (1.0 - I_op)

    # --- base load term (always present) ---
    base = (
        0.2
        + 1.0 * (v_n**2)
        + 0.7 * (P_n**2)
        + 0.3 * (rotor_n**2)
    )

    # --- yaw misalignment term (only relevant in operation) ---
    yaw_term = 0.5 * (yaw_n**2) * v_n * I_op

    # --- parked high-wind term ---
    # more damage if high wind, pitched & parked
    park_term = 1.2 * I_park * (v_n**2) * (0.5 + 0.5 * pitch_n)

    # --- temperature penalty (only when operating) ---
    over_temp = np.maximum(0.0, (T - temp_ref) / 20.0)  # 20°C above ref → 1
    temp_term = 0.3 * (over_temp**2) * I_op

    # --- total synthetic damage ---
    D = base + yaw_term + park_term + temp_term

    return D



In [ ]:
df['damage'] = df.apply(damage_func, axis=1) 

# Neural Network Model

In [ ]:
# First filter out 3 months of data
# set inputs to mean-windspeed and mean-power
# Normalize the inputs
# Split the data with 80% training and 20% testing
# set three hidden layers of 64-64-64
# set activation function to ReLU
# Train the model and interpret training and validation loss
# See if the model performs good on the whole data
# How can we improve the model performance?